In [1]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", module="IPython")

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

print(f"Root dir: {root_dir}")

# Add the root directory to the `PYTHONPATH` 
if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Root dir: /home/xiaotong/Projects/311-service-resolution-prediction
Added the following directory to the PYTHONPATH: /home/xiaotong/Projects/311-service-resolution-prediction
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 2rem; color:#333;">- Part 01: Feature Backfill for NYC 311 Service Requests </span>
1. Choose a specific date and time and download the request information as a CSV file
2. Label the request information, such as complaint_type, descriptor, location_type, agency, borough
3. Create an account on www.hopsworks.ai and get your HOPSWORKS_API_KEY



### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
import datetime
from pathlib import Path
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> STEP 1: Download the Historical NYC 311 Service Requests </span>

In [3]:

import requests
import pandas as pd
from tqdm import tqdm

BASE = "https://data.cityofnewyork.us/resource/erm2-nwe9.json"

APP_TOKEN = ""  
headers = {}
if APP_TOKEN:
    headers["X-App-Token"] = APP_TOKEN

# 2025-09-01 
where = "created_date >= '2025-09-01T00:00:00.000'"

limit = 500     
offset = 0
max_pages = 10000 

all_rows = []

for _ in tqdm(range(max_pages), desc="Downloading"):
    params = {
        "$where": where,
        "$limit": limit,
        "$offset": offset,
        "$order": "created_date ASC, unique_key ASC",
    }

    r = requests.get(BASE, params=params, headers=headers, timeout=120)
    r.raise_for_status()
    batch = r.json()

    if not batch:
        break

    all_rows.extend(batch)
    offset += limit

# 转成 DataFrame
df = pd.DataFrame(all_rows)
print("数据行数, 列数：", df.shape)
output_path = "/content/drive/MyDrive/nyc_311_since_2025_09_01.csv"
df.to_csv(output_path, index=False)
print("已保存到：", output_path)

Downloading:   0%|▏                                                                                                      | 20/10000 [00:18<2:32:17,  1.09it/s]


KeyboardInterrupt: 

---

## <span style='color:#ff5f27'> STEP 2: Read the CSV file </span>

In [3]:
import os
os.getcwd()


'/home/xiaotong/Projects/311-service-resolution-prediction/notebooks/airquality'

In [4]:
# from pathlib import Path
# import pandas as pd
# PROJECT_ROOT = Path(
#     r"C:\Users\20609\Documents\KTHP5P6\scablemachine\lab1\311-service-resolution-prediction"
# )
# DATA_DIR = PROJECT_ROOT / "data"
# df = pd.read_csv(DATA_DIR / "nyc_311_since_2025_09_01.csv")
# df.head()

from pathlib import Path
import pandas as pd

CURRENT_DIR = Path.cwd()
PROJECT_ROOT = CURRENT_DIR.parents[1]

DATA_DIR = PROJECT_ROOT / "data"

df = pd.read_csv(DATA_DIR / "nyc_311_since_2025_09_01.csv")
df.head()





,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,intersection_street_2,vehicle_type,landmark,taxi_pick_up_location,bridge_highway_name,road_ramp,bridge_highway_segment,bridge_highway_direction,due_date,taxi_company_borough
0,66023011,2025-09-01T00:00:00.000,2025-09-02T12:27:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,11421.0,91-50 90 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,66008529,2025-09-01T00:00:10.000,2025-09-01T00:26:39.000,NYPD,New York City Police Department,Noise - Vehicle,Car/Truck Music,Street/Sidewalk,11412.0,116 ROAD,...,FRANCIS LEWIS BOULEVARD,Car,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,66003967,2025-09-01T00:00:11.000,2025-09-01T03:08:50.000,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,10467.0,3320 KOSSUTH AVENUE,...,EAST 210 STREET,NaN,KOSSUTH AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,66006371,2025-09-01T00:00:13.000,2025-09-01T04:22:30.000,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10461.0,1370 BLONDELL AVENUE,...,ROBERTS AVENUE,NaN,BLONDELL AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,66006304,2025-09-01T00:00:15.000,2025-09-01T16:40:30.000,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11370.0,21-38 80 STREET,...,DITMARS BOULEVARD,NaN,80 STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
selected_cols = [
    "created_date",
    "closed_date",
    "agency",
    "agency_name",
    "complaint_type",
    "descriptor",
    "location_type",
    "borough",
    # "latitude",
    # "longitude"
]

df_selected = df[selected_cols].copy()
df_selected["created_date"] = pd.to_datetime(df_selected["created_date"])
df_selected["closed_date"] = pd.to_datetime(df_selected["closed_date"])


In [6]:
df_clean = df_selected.dropna()
df_clean

,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,borough
0,2025-09-01 00:00:00,2025-09-02 12:27:00,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,QUEENS
1,2025-09-01 00:00:10,2025-09-01 00:26:39,NYPD,New York City Police Department,Noise - Vehicle,Car/Truck Music,Street/Sidewalk,QUEENS
2,2025-09-01 00:00:11,2025-09-01 03:08:50,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,BRONX
3,2025-09-01 00:00:13,2025-09-01 04:22:30,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,BRONX
4,2025-09-01 00:00:15,2025-09-01 16:40:30,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,QUEENS
...,...,...,...,...,...,...,...,...
1130430,2025-12-16 01:39:34,2025-12-16 01:58:21,NYPD,New York City Police Department,Noise - Residential,Loud Talking,Residential Building/House,QUEENS
1130434,2025-12-16 01:41:18,2025-12-16 02:05:39,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,BROOKLYN
1130443,2025-12-16 01:45:32,2025-12-16 02:05:00,NYPD,New York City Police Department,Noise - Residential,Loud Talking,Residential Building/House,MANHATTAN
1130448,2025-12-16 01:46:54,2025-12-16 01:57:45,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,QUEENS


In [7]:
import holidays
import pytz

us_holidays = holidays.US()
ny_tz = pytz.timezone("America/New_York") #NY time zone 
df_clean = df_clean.copy()

df_clean["created_date"] = pd.to_datetime(df_clean["created_date"])
df_clean["created_date_ny"] = df_clean["created_date"].dt.tz_localize("UTC").dt.tz_convert(ny_tz)

df_clean["date"] = df_clean["created_date_ny"].dt.date
df_clean["hour"] = df_clean["created_date_ny"].dt.hour
df_clean["weekday"] = df_clean["created_date_ny"].dt.weekday  # Monday=0

df_clean["is_holiday"] = df_clean["date"].apply(
    lambda x: 1 if x in us_holidays else 0
)

df_clean["is_work_day"] = (
    (df_clean["weekday"] < 5) & (df_clean["is_holiday"] == 0)
).astype(int)

df_clean["is_work_hours"] = (
    (df_clean["is_work_day"] == 1) &
    (df_clean["hour"] >= 9) &
    (df_clean["hour"] < 17)
).astype(int)

cols_to_drop = [
    "created_date_ny",
    "date",
    "hour",
    "weekday",
    "is_holiday"
]

df_clean = df_clean.drop(columns=cols_to_drop)
df_nyc_311 = df_clean
df_nyc_311

,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,borough,is_work_day,is_work_hours
0,2025-09-01 00:00:00,2025-09-02 12:27:00,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,QUEENS,0,0
1,2025-09-01 00:00:10,2025-09-01 00:26:39,NYPD,New York City Police Department,Noise - Vehicle,Car/Truck Music,Street/Sidewalk,QUEENS,0,0
2,2025-09-01 00:00:11,2025-09-01 03:08:50,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,BRONX,0,0
3,2025-09-01 00:00:13,2025-09-01 04:22:30,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,BRONX,0,0
4,2025-09-01 00:00:15,2025-09-01 16:40:30,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,QUEENS,0,0
...,...,...,...,...,...,...,...,...,...,...
1130430,2025-12-16 01:39:34,2025-12-16 01:58:21,NYPD,New York City Police Department,Noise - Residential,Loud Talking,Residential Building/House,QUEENS,1,0
1130434,2025-12-16 01:41:18,2025-12-16 02:05:39,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,BROOKLYN,1,0
1130443,2025-12-16 01:45:32,2025-12-16 02:05:00,NYPD,New York City Police Department,Noise - Residential,Loud Talking,Residential Building/House,MANHATTAN,1,0
1130448,2025-12-16 01:46:54,2025-12-16 01:57:45,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,QUEENS,1,0


## Hopsworks API Key
You need to have registered an account on app.hopsworks.ai.

Save the HOPSWORKS_API_KEY  to ~/.env file in the root directory of your project

 * mv .env.example .env
 * edit .env

In the .env file, update HOPSWORKS_API_KEY:

`HOPSWORKS_API_KEY="put API KEY value in this string"`


In [8]:
project = hopsworks.login(
    api_key_value="YOUR_API_KEY",  
    host="c.app.hopsworks.ai", 
    project="Lab1_xiaotong",  
    engine="python"  
)

2026-01-09 12:12:54,630 INFO: Initializing external client
2026-01-09 12:12:54,631 INFO: Base URL: https://c.app.hopsworks.ai:443


2026-01-09 12:12:56,107 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286339


## Daily Weather for Five NYC Regions (each day 5 rows)

In [9]:
import pandas as pd

BOROUGH_COORDS = {
    "MANHATTAN": (40.7829, -73.9654),
    "BROOKLYN": (40.6928, -73.9903),
    "QUEENS": (40.7769, -73.8740),
    "BRONX": (40.8506, -73.8769),
    "STATEN ISLAND": (40.6437, -74.0736),
}


In [10]:
import requests
import pandas as pd

# - temperature (average over the day)
# - precipitation (the total over the day)
# - wind speed (average over the day)

TZ = "America/New_York"
START_DATE = "2025-09-01"
END_DATE = "2025-12-17"
HOURLY_VARS = ["temperature_2m", "precipitation", "wind_speed_10m"]


def _fetch_open_meteo_hourly(lat: float, lon: float) -> pd.DataFrame:
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": float(lat),
        "longitude": float(lon),
        "start_date": START_DATE,
        "end_date": END_DATE,
        "hourly": ",".join(HOURLY_VARS),
        "timezone": TZ,
    }
    r = requests.get(url, params=params, timeout=120)
    r.raise_for_status()
    data = r.json()
    hourly = (data or {}).get("hourly") or {}
    if "time" not in hourly:
        return pd.DataFrame(columns=["datetime"] + HOURLY_VARS)

    dfh = pd.DataFrame(hourly)
    dfh["datetime"] = pd.to_datetime(dfh["time"], errors="coerce")
    return dfh.drop(columns=["time"])


_frames = []
for borough, (lat, lon) in BOROUGH_COORDS.items():
    dfh = _fetch_open_meteo_hourly(lat, lon)
    if dfh.empty:
        continue

    dfh["borough"] = borough
    dfh["date"] = dfh["datetime"].dt.date

    daily = (
        dfh.groupby(["borough", "date"], as_index=False)
        .agg(
            temperature_mean=("temperature_2m", "mean"),
            precipitation_sum=("precipitation", "sum"),
            wind_speed_mean=("wind_speed_10m", "mean"),
        )
    )
    _frames.append(daily)

borough_weather_daily = (
    pd.concat(_frames, ignore_index=True)
    if _frames
    else pd.DataFrame(columns=["borough", "date", "temperature_mean", "precipitation_sum", "wind_speed_mean"])
)

_all_dates = pd.date_range(START_DATE, END_DATE, freq="D").date
_grid = pd.MultiIndex.from_product(
    [list(BOROUGH_COORDS.keys()), _all_dates],
    names=["borough", "date"],
).to_frame(index=False)

borough_weather_daily = _grid.merge(borough_weather_daily, on=["borough", "date"], how="left")
borough_weather_daily["latitude"] = borough_weather_daily["borough"].map(lambda b: BOROUGH_COORDS[b][0])
borough_weather_daily["longitude"] = borough_weather_daily["borough"].map(lambda b: BOROUGH_COORDS[b][1])
borough_weather_daily = borough_weather_daily.sort_values(["date", "borough"]).reset_index(drop=True)

borough_weather_daily.head(200)

,borough,date,temperature_mean,precipitation_sum,wind_speed_mean,latitude,longitude
0,BRONX,2025-09-01,21.204167,0.0,8.066667,40.8506,-73.8769
1,BROOKLYN,2025-09-01,21.816667,0.0,8.625000,40.6928,-73.9903
2,MANHATTAN,2025-09-01,21.479167,0.0,8.325000,40.7829,-73.9654
3,QUEENS,2025-09-01,21.379167,0.0,8.300000,40.7769,-73.8740
4,STATEN ISLAND,2025-09-01,21.308333,0.1,8.837500,40.6437,-74.0736
...,...,...,...,...,...,...,...
195,BRONX,2025-10-10,11.291667,0.0,6.954167,40.8506,-73.8769
196,BROOKLYN,2025-10-10,11.758333,0.0,8.095833,40.6928,-73.9903
197,MANHATTAN,2025-10-10,10.995833,0.0,7.016667,40.7829,-73.9654
198,QUEENS,2025-10-10,11.420833,0.0,6.416667,40.7769,-73.8740


## Uploading new data to the Feature Store

In [11]:
fs = project.get_feature_store() 

In [12]:
service_request_fg = fs.get_or_create_feature_group(
    name="nyc_311_service_request",
    version=1,
    description="NYC 311 service requests",
    primary_key=[
        "created_date",
        "agency",
        "complaint_type",
        "borough"
    ]
)
service_request_fg.insert(
    df_nyc_311,
    write_options={
        "wait_for_job": True
    }
)


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1286339/fs/1284167/fg/1911249


Uploading Dataframe: 100.00% |████████████████████████| Rows 910500/910500 | Elapsed Time: 00:24 | Remaining Time: 00:00


Launching job: nyc_311_service_request_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286339/jobs/named/nyc_311_service_request_1_offline_fg_materialization/executions
2026-01-09 13:19:15,064 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2026-01-09 13:19:18,248 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2026-01-09 13:22:10,230 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2026-01-09 13:22:10,409 INFO: Waiting for log aggregation to finish.
2026-01-09 13:22:35,854 INFO: Execution finished successfully.


(Job('nyc_311_service_request_1_offline_fg_materialization', 'SPARK'), None)

In [14]:
weather_fg = fs.get_or_create_feature_group(
    name="nyc_weather_daily",
    version=1,
    primary_key=["borough", "date"],
    event_time="date",
    description="Daily borough-level weather features for NYC (mean temperature, total precipitation, mean wind speed)."
)
weather_fg.insert(
    borough_weather_daily,
    write_options={
        "wait_for_job": True
    }
)


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1286339/fs/1284167/fg/1878460


Uploading Dataframe: 100.00% |████████████████████████████████████████████████████████████████████| Rows 540/540 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: nyc_weather_daily_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286339/jobs/named/nyc_weather_daily_1_offline_fg_materialization/executions
2025-12-27 21:46:51,598 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-12-27 21:46:54,767 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-12-27 21:46:57,940 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-12-27 21:49:08,289 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-12-27 21:49:08,466 INFO: Waiting for log aggregation to finish.
2025-12-27 21:49:23,858 INFO: Execution finished successfully.


(Job('nyc_weather_daily_1_offline_fg_materialization', 'SPARK'), None)

In [14]:

import pandas as pd
TZ = "America/New_York"

weather_small = borough_weather_daily.copy()
weather_small["borough_upper"] = weather_small["borough"].astype(str).str.upper().str.strip()
weather_small["service_date"] = pd.to_datetime(weather_small["date"], errors="coerce").dt.date
weather_small = weather_small[[
    "borough_upper",
    "service_date",
    "temperature_mean",
    "precipitation_sum",
    "wind_speed_mean",
]].rename(
    columns={
        "temperature_mean": "weather_temperature_mean",
        "precipitation_sum": "weather_precipitation_sum",
        "wind_speed_mean": "weather_wind_speed_mean",
    }
)

df_final = df_clean.copy()
df_final["borough_upper"] = df_final["borough"].astype(str).str.upper().str.strip()

_created_naive = pd.to_datetime(df_final["created_date"], errors="coerce")
_service_date_local = _created_naive.dt.date

_created_utc = pd.to_datetime(df_final["created_date"], errors="coerce", utc=True)
_service_date_utc_to_ny = _created_utc.dt.tz_convert(TZ).dt.date

_sample_n = 100_000
_sample = df_final[["borough_upper"]].copy()
_sample["d_local"] = _service_date_local
_sample["d_utc_to_ny"] = _service_date_utc_to_ny
_sample = _sample.dropna(subset=["borough_upper"])
if len(_sample) > _sample_n:
    _sample = _sample.sample(_sample_n, random_state=42)

_tmp_local = _sample.merge(
    weather_small[["borough_upper", "service_date"]],
    left_on=["borough_upper", "d_local"],
    right_on=["borough_upper", "service_date"],
    how="left",
)
_tmp_utc = _sample.merge(
    weather_small[["borough_upper", "service_date"]],
    left_on=["borough_upper", "d_utc_to_ny"],
    right_on=["borough_upper", "service_date"],
    how="left",
)

rate_local = _tmp_local["service_date"].notna().mean() if len(_tmp_local) else 0.0
rate_utc = _tmp_utc["service_date"].notna().mean() if len(_tmp_utc) else 0.0

use_utc_to_ny = rate_utc > rate_local
print(f"Join match rate (sample): local={rate_local:.3f}, utc_to_ny={rate_utc:.3f} -> using {'utc_to_ny' if use_utc_to_ny else 'local'}")

df_final["service_date"] = _service_date_utc_to_ny if use_utc_to_ny else _service_date_local
df_final = df_final.merge(weather_small, on=["borough_upper", "service_date"], how="left")
_df_weather_cols = ["weather_temperature_mean", "weather_precipitation_sum", "weather_wind_speed_mean"]
print("Weather non-null rate:", df_final[_df_weather_cols].notna().mean().to_dict())
df_final = df_final.drop(columns=["borough_upper", "service_date"])

df_final[_df_weather_cols].head()

Join match rate (sample): local=1.000, utc_to_ny=0.998 -> using local
Weather non-null rate: {'weather_temperature_mean': 0.9996265788028555, 'weather_precipitation_sum': 0.9996265788028555, 'weather_wind_speed_mean': 0.9996265788028555}


,weather_temperature_mean,weather_precipitation_sum,weather_wind_speed_mean
0,21.379167,0.0,8.300000
1,21.379167,0.0,8.300000
2,21.204167,0.0,8.066667
3,21.204167,0.0,8.066667
4,21.379167,0.0,8.300000


In [15]:
OUTPUT_CSV = "nyc_weather.csv"

borough_weather_daily.to_csv(
    OUTPUT_CSV,
    index=False,
    encoding="utf-8-sig",  
)

In [16]:
import pandas as pd

TZ = "America/New_York"

weather_small = borough_weather_daily.copy()
weather_small["borough_upper"] = weather_small["borough"].astype(str).str.upper().str.strip()
weather_small["service_date"] = pd.to_datetime(weather_small["date"], errors="coerce").dt.date
weather_small = weather_small[[
    "borough_upper",
    "service_date",
    "temperature_mean",
    "precipitation_sum",
    "wind_speed_mean",
]].rename(
    columns={
        "temperature_mean": "weather_temperature_mean",
        "precipitation_sum": "weather_precipitation_sum",
        "wind_speed_mean": "weather_wind_speed_mean",
    }
)

df_final = df_clean.copy()
df_final["borough_upper"] = df_final["borough"].astype(str).str.upper().str.strip()

_created_naive = pd.to_datetime(df_final["created_date"], errors="coerce")
_service_date_local = _created_naive.dt.date

_created_utc = pd.to_datetime(df_final["created_date"], errors="coerce", utc=True)
_service_date_utc_to_ny = _created_utc.dt.tz_convert(TZ).dt.date

_sample_n = 100_000
_sample = df_final[["borough_upper"]].copy()
_sample["d_local"] = _service_date_local
_sample["d_utc_to_ny"] = _service_date_utc_to_ny
_sample = _sample.dropna(subset=["borough_upper"])
if len(_sample) > _sample_n:
    _sample = _sample.sample(_sample_n, random_state=42)

_tmp_local = _sample.merge(
    weather_small[["borough_upper", "service_date"]],
    left_on=["borough_upper", "d_local"],
    right_on=["borough_upper", "service_date"],
    how="left",
)
_tmp_utc = _sample.merge(
    weather_small[["borough_upper", "service_date"]],
    left_on=["borough_upper", "d_utc_to_ny"],
    right_on=["borough_upper", "service_date"],
    how="left",
)

rate_local = _tmp_local["service_date"].notna().mean() if len(_tmp_local) else 0.0
rate_utc = _tmp_utc["service_date"].notna().mean() if len(_tmp_utc) else 0.0

use_utc_to_ny = rate_utc > rate_local
print(f"Join match rate (sample): local={rate_local:.3f}, utc_to_ny={rate_utc:.3f} -> using {'utc_to_ny' if use_utc_to_ny else 'local'}")
df_final["service_date"] = _service_date_utc_to_ny if use_utc_to_ny else _service_date_local
df_final = df_final.merge(weather_small, on=["borough_upper", "service_date"], how="left")
_df_weather_cols = ["weather_temperature_mean", "weather_precipitation_sum", "weather_wind_speed_mean"]
print("Weather non-null rate:", df_final[_df_weather_cols].notna().mean().to_dict())
df_final = df_final.drop(columns=["borough_upper", "service_date"])
df_final.head(200)


Join match rate (sample): local=1.000, utc_to_ny=0.998 -> using local
Weather non-null rate: {'weather_temperature_mean': 0.9996265788028555, 'weather_precipitation_sum': 0.9996265788028555, 'weather_wind_speed_mean': 0.9996265788028555}


,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,borough,is_work_day,is_work_hours,weather_temperature_mean,weather_precipitation_sum,weather_wind_speed_mean
0,2025-09-01 00:00:00,2025-09-02 12:27:00,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,QUEENS,0,0,21.379167,0.0,8.300000
1,2025-09-01 00:00:10,2025-09-01 00:26:39,NYPD,New York City Police Department,Noise - Vehicle,Car/Truck Music,Street/Sidewalk,QUEENS,0,0,21.379167,0.0,8.300000
2,2025-09-01 00:00:11,2025-09-01 03:08:50,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,BRONX,0,0,21.204167,0.0,8.066667
3,2025-09-01 00:00:13,2025-09-01 04:22:30,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,BRONX,0,0,21.204167,0.0,8.066667
4,2025-09-01 00:00:15,2025-09-01 16:40:30,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,QUEENS,0,0,21.379167,0.0,8.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2025-09-01 00:18:40,2025-09-01 02:26:30,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,BROOKLYN,0,0,21.816667,0.0,8.625000
196,2025-09-01 00:18:42,2025-09-01 00:41:17,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,BROOKLYN,0,0,21.816667,0.0,8.625000
197,2025-09-01 00:18:49,2025-09-01 01:17:40,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,QUEENS,0,0,21.379167,0.0,8.300000
198,2025-09-01 00:18:55,2025-09-01 01:14:52,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,BRONX,0,0,21.204167,0.0,8.066667


## Save merged data

In [17]:
df_final.to_csv("df_final.csv", index=False, encoding="utf-8-sig")

In [19]:
import pandas as pd

df = df_final.copy()
df["created_date"] = pd.to_datetime(df["created_date"], errors="coerce")
df["closed_date"]  = pd.to_datetime(df["closed_date"], errors="coerce")
df = df.dropna(subset=["created_date", "closed_date"]).copy()
df["resolution_hours"] = (df["closed_date"] - df["created_date"]).dt.total_seconds() / 3600
df = df[df["resolution_hours"] >= 0].copy()
df["label_48h"] = (df["resolution_hours"] <= 48).astype(int)
df[["resolution_hours","label_48h"]].head()


,resolution_hours,label_48h
0,36.450000,1
1,0.441389,1
2,3.144167,1
3,4.371389,1
4,16.670833,1


In [20]:
df["created_hour"] = df["created_date"].dt.hour
df["created_weekday"] = df["created_date"].dt.weekday
df["created_month"] = df["created_date"].dt.month
df["is_weekend"] = (df["created_weekday"] >= 5).astype(int)
